# NPS Corporate

In [1]:
#import pandas as pd
import re
from IPython.display import Markdown, HTML, display, FileLink
from ThaiTextPrepKit import polars_pretextkit as preprocess, typo_patterns as TYPO
import polars as pl
import pythainlp
#import pandas as pd
#from openpyxl import Workbook
#from openpyxl.utils.dataframe import dataframe_to_rows
#from openpyxl.styles import PatternFill
#from bs4 import BeautifulSoup

In [2]:
'''def highlight_patterns(patterns, text, highlight_color="#FFFF00"):
    """
    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.

    :param patterns: A list of tuples containing regex patterns and their replacements.
    :param text: The text to search within.
    :param highlight_color: The background color to use for highlighting (default is yellow).
    :return: The text with highlighted matches.
    """
    def add_highlight(match):
        return f'<span style="background-color: {highlight_color};">{match.group(0)}</span>'
    
    highlighted_text = text
    for pattern, replacement in patterns:
        highlighted_text = re.sub(pattern, add_highlight, highlighted_text)
    
    return highlighted_text'''

'def highlight_patterns(patterns, text, highlight_color="#FFFF00"):\n    """\n    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.\n\n    :param patterns: A list of tuples containing regex patterns and their replacements.\n    :param text: The text to search within.\n    :param highlight_color: The background color to use for highlighting (default is yellow).\n    :return: The text with highlighted matches.\n    """\n    def add_highlight(match):\n        return f\'<span style="background-color: {highlight_color};">{match.group(0)}</span>\'\n    \n    highlighted_text = text\n    for pattern, replacement in patterns:\n        highlighted_text = re.sub(pattern, add_highlight, highlighted_text)\n    \n    return highlighted_text'

In [3]:
def highlight_patterns(patterns, text, html=True, highlight_color_replace="#FFFF00", highlight_color_match="#00FFFF"):
    """
    Highlights all occurrences of the given regex patterns in the text using HTML <span> tags with background color.

    :param patterns: A list of tuples containing regex patterns and their replacements.
    :param text: The text to search within.
    :param highlight_color_replace: The background color to use for replacements (default is yellow).
    :param highlight_color_match: The background color to use for matches without replacement (default is blue).
    :return: The text with highlighted matches.
    """
    def add_highlight(match, color):
        return f'<span style="background-color: {color};">{match.group(0)}</span>' if html else f'<typo>{match.group(0)}</typo>'
    
    highlighted_text = text
    for pattern, replacement in patterns:
        matches = list(re.finditer(pattern, text))
        replacement = replacement.lstrip('<IGNORE>').rstrip('</IGNORE>')
        print(replacement)
        if matches:
            for match in matches:
                if replacement in match.group(0):
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_match))
                else:
                    highlighted_text = highlighted_text.replace(match.group(0), add_highlight(match, highlight_color_replace))
    
    return highlighted_text

In [4]:
patterns = TYPO.corp_patterns

In [25]:
text = "I love using แอ๊บ and แอปพลิเคชัน application on my apple device."

highlighted_text = highlight_patterns(patterns, text, html=False)

In [6]:
highlighted_text

'I love using <typo>แอ๊บ</typo> and <typo>แอปพลิเคชัน</typo> <typo>application</typo> on my <typo>apple</typo> device<typo>.</typo>'

In [7]:
display(Markdown(highlighted_text))

I love using <typo>แอ๊บ</typo> and <typo>แอปพลิเคชัน</typo> <typo>application</typo> on my <typo>apple</typo> device<typo>.</typo>

In [8]:
highlighted_text

'I love using <typo>แอ๊บ</typo> and <typo>แอปพลิเคชัน</typo> <typo>application</typo> on my <typo>apple</typo> device<typo>.</typo>'

In [9]:
def get_highlight_texts(patterns, texts: list) -> None:
    html_text = []
    for text in texts:
        html_text.append(highlight_patterns(patterns, text))
    return html_text

In [10]:
text_cols = [
    'Recommend1-Bank1',
    'Recommend2-Bank1',
    'Recommend3-Bank1',
    'NotRecommend1-Bank1',
    'NotRecommend2-Bank1',
    'NotRecommend3-Bank1',
    'Recommend1-Bank2',
    'Recommend2-Bank2',
    'Recommend3-Bank2',
    'NotRecommend1-Bank2',
    'NotRecommend2-Bank2',
    'NotRecommend3-Bank2',
    'Recommend1-Bank3',
    'Recommend2-Bank3',
    'Recommend3-Bank3',
    'NotRecommend1-Bank3',
    'NotRecommend2-Bank3',
    'NotRecommend3-Bank3',
]

In [11]:
path = r"C:\Users\patom\OneDrive\Documents\Repo\DataPrepKit\Samples\NPS Reason HY'24-textprocessing test1.xlsx"
#path = "/Users/pa/Documents/GitHub Repository Clone/DataPrepKit/Samples/NPS Reason HY'24-textprocessing test1.xlsx"

df = pl.read_excel(path,
                   engine='calamine')

concat_df = df.with_columns(
    pl.concat_list(text_cols).alias('text')
)

In [12]:
concat_df = concat_df.explode('text')
concat_df = concat_df.drop_nulls('text')

In [13]:
original_texts = concat_df.get_column('text').to_list()

In [14]:
"""texts = [
    'แอปพลิเคชันดี',
    'แอพลิเคชั่นดี',
    'internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ',
    'internet banking',
    'net banking',
    'อินเตอร์เน็ต',
    'อินเตอร์เน็ตแบงค์กิ้ง',
    'internet แบงกิ้ง',
    'baht net',
    'bath net',
    'net',
    'I love using แอ๊บ and แอปพลิเคชัน application on my apple device.',
]"""
html_texts = get_highlight_texts(patterns, original_texts)

for text in html_texts[:5]:
    display(Markdown(text))

<span style="background-color: #00FFFF;">พนักงาน</span>ฝ่าย <span style="background-color: #FFFF00;">Call Center</span> ให้คำปรึกษาได้ตลอด

<span style="background-color: #00FFFF;">พนักงาน</span> <span style="background-color: #FFFF00;">Call Center</span> ให้ความรู้ ให้ข้อมูลถูกต้อง

<span style="background-color: #00FFFF;">พนักงาน</span> <span style="background-color: #FFFF00;">Call Center</span> ใส่ใจและติดตามงานให้ดี

ระบบ<span style="background-color: #00FFFF;">แบงก์</span>กิ้ง<span style="background-color: #00FFFF;">ใช้งาน</span>ยาก เข้าเมนูกลับไปกลับมา

ระบบ<span style="background-color: #00FFFF;">แบงก์</span>กิ้งเมื่อ<span style="background-color: #00FFFF;">อัปเดต</span>ระบบใหม่ จะนำมาใช้บนมือถือยาก ตัวหนังสือจะเป็นแนวตั้ง ไม่เป็นแนวนอนเหมือนใช้บนโน๊ตบุ๊ค

In [15]:
import re
#skip_patterns = [re.compile(rf'(เบอ[ร]*[์]*โท[ร]*[สศ]ั[พบ][ท]*[์]*)|(เบอ[ร์]*โท[ร]*)|(เบอ(?!ะ)[ร]*[์]*)', re.IGNORECASE),]

In [16]:
series = pl.Series(original_texts)
pre_series = preprocess.preprocess_text_batches(series=series,
                                   custom_dict=None,
                                   keep_stopwords=True,
                                   keep_format=True,
                                   return_token_list=False,
                                   lower_case=False,
                                   include_pattern='[/()]',
                                   skip_patterns=None,
                                   patterns=patterns)

for t in pre_series[:5]:
    print(t)

พนักงานฝ่าย คอลเซ็นเตอร์ ให้คำปรึกษาได้ตลอด
พนักงาน คอลเซ็นเตอร์ ให้ความรู้ ให้ข้อมูลถูกต้อง
พนักงาน คอลเซ็นเตอร์ ใส่ใจและติดตามงานให้ดี
ระบบแบงก์กิ้งใช้งานยาก เข้าเมนูกลับไปกลับมา
ระบบแบงก์กิ้งเมื่ออัปเดตระบบใหม่ จะนำมาใช้บนมือถือยาก ตัวหนังสือจะเป็นแนวตั้ง ไม่เป็นแนวนอนเหมือนใช้บนโน๊ตบุ๊ค


In [17]:
'''def insert_typo_token_pythainlp_engine(texts_list: list, html: bool=True) -> list:
  inserted_token_texts = []
  for text in texts_list:
    text_tokens = pythainlp.word_tokenize(text, keep_whitespace=True, join_broken_num=True)
    corrected_text_tokens = [pythainlp.correct(token, engine='pn') if token != ' ' else token for token in text_tokens]
    text_with_typo_token = []
    for (original, corrected) in zip(text_tokens, corrected_text_tokens):
      if original == corrected:
        text_with_typo_token.append(original)
      else:
        token_str = f'<span style="background-color: #FFFF00;">{original}</span>' if html else f'<typo>{original}</typo>'
        text_with_typo_token.append(token_str)
    inserted_token_texts.append(''.join(text_with_typo_token))
  return inserted_token_texts'''

def insert_typo_token_pythainlp_engine(texts_list: list, html: bool=True) -> list:
  raw_inserted_token_texts = []
  pre_inserted_token_texts = []
  for text in texts_list:
    text_tokens = pythainlp.word_tokenize(text, keep_whitespace=True, join_broken_num=True)
    corrected_text_tokens = [pythainlp.correct(token, engine='pn') if token != ' ' else token for token in text_tokens]
    raw_text_with_typo_token = []
    pre_text_with_typo_token = []
    for (original, corrected) in zip(text_tokens, corrected_text_tokens):
      if original == corrected:
        raw_text_with_typo_token.append(original)
        pre_text_with_typo_token.append(original)
      else:
        raw_token_str = f'<span style="background-color: #FFFF00;">{original}</span>' if html else f'<typo>{original}</typo>'
        pre_token_str = f'<span style="background-color: #00FFFF;">{corrected}</span>' if html else f'<typo>{corrected}</typo>'
        raw_text_with_typo_token.append(raw_token_str)
        pre_text_with_typo_token.append(pre_token_str)
    raw_inserted_token_texts.append(''.join(raw_text_with_typo_token))
    pre_inserted_token_texts.append(''.join(pre_text_with_typo_token))
  return raw_inserted_token_texts, pre_inserted_token_texts

In [18]:


'''def generate_html_table(original_texts, preprocess_texts):
    """
    Generate an HTML table with 'original_text' and 'preprocess_text' columns.

    Args:
    - original_texts (list): List of original texts.
    - preprocess_texts (list): List of preprocessed texts.

    Returns:
    - html_content (str): String containing the HTML table.
    """
    html_content = "<table border='1'>\n"
    html_content += "<tr><th>original_text</th><th>preprocess_text</th></tr>\n"

    for original, preprocess in zip(original_texts, preprocess_texts):
        html_content += f"<tr><td>{original}</td><td>{preprocess}</td></tr>\n"

    html_content += "</table>"
    
    return html_content'''


'def generate_html_table(original_texts, preprocess_texts):\n    """\n    Generate an HTML table with \'original_text\' and \'preprocess_text\' columns.\n\n    Args:\n    - original_texts (list): List of original texts.\n    - preprocess_texts (list): List of preprocessed texts.\n\n    Returns:\n    - html_content (str): String containing the HTML table.\n    """\n    html_content = "<table border=\'1\'>\n"\n    html_content += "<tr><th>original_text</th><th>preprocess_text</th></tr>\n"\n\n    for original, preprocess in zip(original_texts, preprocess_texts):\n        html_content += f"<tr><td>{original}</td><td>{preprocess}</td></tr>\n"\n\n    html_content += "</table>"\n    \n    return html_content'

In [19]:
'''def generate_html_table(*args):
    """
    Generate an HTML table with columns for each text list provided.

    Args:
    - *args (list): Variable number of lists, each containing texts.

    Returns:
    - html_content (str): String containing the HTML table.
    """
    num_columns = len(args)
    headers = [f"text_list_{i+1}" for i in range(num_columns)]
    
    html_content = "<table border='1'>\n"
    html_content += "<tr>" + "".join(f"<th>{header}</th>" for header in headers) + "</tr>\n"
    
    # Find the maximum length among the provided lists to handle uneven lengths
    max_length = max(len(lst) for lst in args)
    
    for i in range(max_length):
        html_content += "<tr>"
        for lst in args:
            cell_content = lst[i] if i < len(lst) else ""
            html_content += f"<td>{cell_content}</td>"
        html_content += "</tr>\n"

    html_content += "</table>"
    
    return html_content'''


'def generate_html_table(*args):\n    """\n    Generate an HTML table with columns for each text list provided.\n\n    Args:\n    - *args (list): Variable number of lists, each containing texts.\n\n    Returns:\n    - html_content (str): String containing the HTML table.\n    """\n    num_columns = len(args)\n    headers = [f"text_list_{i+1}" for i in range(num_columns)]\n    \n    html_content = "<table border=\'1\'>\n"\n    html_content += "<tr>" + "".join(f"<th>{header}</th>" for header in headers) + "</tr>\n"\n    \n    # Find the maximum length among the provided lists to handle uneven lengths\n    max_length = max(len(lst) for lst in args)\n    \n    for i in range(max_length):\n        html_content += "<tr>"\n        for lst in args:\n            cell_content = lst[i] if i < len(lst) else ""\n            html_content += f"<td>{cell_content}</td>"\n        html_content += "</tr>\n"\n\n    html_content += "</table>"\n    \n    return html_content'

In [20]:
def generate_html_table(*args):
    """
    Generate an HTML table with columns for each text list provided, using the given names as headers.

    Args:
    - *args (tuple): Variable number of tuples, each containing a list of texts and a corresponding name.

    Returns:
    - html_content (str): String containing the HTML table.
    """
    headers = [name for _, name in args]
    
    html_content = "<table border='1'>\n"
    html_content += "<tr>" + "".join(f"<th>{header}</th>" for header in headers) + "</tr>\n"
    
    # Find the maximum length among the provided lists to handle uneven lengths
    max_length = max(len(text_list) for text_list, _ in args)
    
    for i in range(max_length):
        html_content += "<tr>"
        for text_list, _ in args:
            cell_content = text_list[i] if i < len(text_list) else ""
            html_content += f"<td>{cell_content}</td>"
        html_content += "</tr>\n"

    html_content += "</table>"
    
    return html_content

In [21]:
# pythainlp pn model engine
#pn_raw_token_texts, pn_pre_token_texts = insert_typo_token_pythainlp_engine(original_texts, html=True)

In [22]:
# Example lists (replace with your data)
'''original_texts = ['<span style="background-color: #00FFFF;">แอปพลิเคชัน</span>ดี',
                  '<span style="background-color: #FFFF00;">แอพลิเคชั่น</span>ดี',
                  'internet banking <span style="background-color: #00FFFF;">ใช้งาน</span>ง่าย มีคู่มือให้ดูได้จากหน้าเว็บ',]

preprocess_texts = ['แอปพลิเคชันดี',
                    'แอปพลิเคชันดี',
                    'internet banking ใช้งานง่าย มีคู่มือให้ดูได้จากหน้าเว็บ']'''

# Generate HTML table
html_table = generate_html_table((html_texts, 'raw text'), 
                                 (get_highlight_texts(patterns, pre_series.to_list()), "Pa's regex pattern"),
                                 #(pn_raw_token_texts, 'pn engine detection'),
                                 #(pn_pre_token_texts, 'pn engine correction')
                                 )

# Display HTML in Jupyter Notebook
from IPython.display import HTML, display
#display(HTML(html_table))

In [23]:
# Create a downloadable link for the HTML content
with open('table3.html', 'w', encoding='utf-8') as f:
    f.write(html_table)

FileLink('table3.html')

c:\Users\patom\OneDrive\Documents\Repo\DataPrepKit\table3.html

In [24]:
'<IGNORE>เบอร์</IGNORE>'.lstrip('<IGNORE>').rstrip('</IGNORE>')

'เบอร์'